In [ ]:
#| default_exp classes.DomoBootstrap

In [ ]:
#| export
from fastcore.basics import patch, patch_to



In [ ]:
#| exporti
from dataclasses import dataclass, field

import httpx

import domolibrary.classes.DomoDataset as dmds
import domolibrary.classes.DomoPage as dmpg

import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.Logger as lc
import domolibrary.routes.bootstrap as bootstrap_routes


In [ ]:
#| export
@dataclass
class DomoBootstrap_Feature:
    id: int
    name: str
    label: str
    type: str
    purchased: bool
    enabled: bool

    @classmethod
    def _from_json_bootstrap(cls, json_obj: dict):
        dd = util_dd.DictDot(json_obj)

        bsf = cls(
            id=dd.id,
            name=dd.name,
            label=dd.label,
            type=dd.type,
            purchased=dd.purchased,
            enabled=dd.enabled
        )
        return bsf

In [ ]:
#| export
@dataclass
class DomoBootstrap:
    auth : dmda.DomoAuth = field(repr = False)
    page_ls : list[dmpg.DomoPage] = field(default = None)
    feature_ls : list[DomoBootstrap_Feature] = field(default = None)
    

In [ ]:
# | export
@patch_to(DomoBootstrap)
async def get_all(self: DomoBootstrap,
                  auth: dmda.DomoAuth = None, debug_api: bool = False):
    
    auth = auth or self.auth

    res =  await bootstrap_routes.get_bootstrap(auth=auth, debug_api=debug_api)

    return res.response


#### sample implementation of get_all

In [ ]:
import os

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-dojo', domo_password = os.environ['DOJO_PASSWORD'], domo_username = 'jae@onyxreporting.com')

bs = DomoBootstrap( auth = full_auth)

bsr = await bs.get_all()
bsr.keys()

dict_keys(['notifierConfig', 'currentUser', 'data'])

In [ ]:
# | export
@patch_to(DomoBootstrap)
async def get_pages(self: DomoBootstrap,
                    auth: dmda.DomoAuth = None, return_raw : bool = False, debug_api: bool = False) -> list[dmpg.DomoPage]:

    auth = auth or self.auth

    res = await bootstrap_routes.get_bootstrap_pages(auth=auth,
                                                     debug_api=debug_api)
    
    if return_raw:
        return res.response
        
    if not res.is_success:
        return None
    
    page_ls = res.response

    self.page_ls =  [dmpg.DomoPage._from_bootstrap(page_obj, auth = auth) for page_obj in page_ls]

    return self.page_ls


#### sample implementation of get_pages

In [ ]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-dojo', 
    domo_password=os.environ['DOJO_PASSWORD'],
    domo_username='jae@onyxreporting.com')

bs = DomoBootstrap(auth=full_auth)

page_ls = await bs.get_pages(return_raw = False)

# bs.page_ls
pd.DataFrame(page_ls[10:14]).drop(columns = ['auth'] )


DictDot(id=1814479647, type='GROUP', displayName='Admin Test')


,id,title,parent_page_id,owners,cards,collections,children
0,790951325,Coronavirus (COVID-19) Dashboard,None,"[{'id': 1345408759, 'display_name': 'Alexis Lo...",[],[],"[{'id': '1289295627', 'title': 'Dataset Schema..."
1,1891382078,Fun Stuff,None,"[{'id': 1345408759, 'display_name': 'Alexis Lo...",[],[],"[{'id': '1124898205', 'title': 'Quotes', 'pare..."
2,545237196,Demo Apps,None,"[{'id': 826020125, 'display_name': 'Ritwik', '...",[],[],[]
3,400620119,Data Science Workshop,None,"[{'id': 1504505648, 'display_name': 'Ken Loren...",[],[],[]


In [ ]:
#| export
@patch_to(DomoBootstrap)
async def get_features(self : DomoBootstrap,
                        auth: dmda.DomoAuth = None,
                        debug_api: bool = False,
                        return_raw:bool = False,
                        session: httpx.AsyncClient = None, 
                        ):
    
    auth = auth or self.auth

    res = await bootstrap_routes.get_bootstrap_features(auth=auth, session=session, debug_api=debug_api)

    if return_raw:
        return res

    feature_list = [DomoBootstrap_Feature._from_json_bootstrap(
        json_obj) for json_obj in res.response]

    return feature_list

In [ ]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-dojo',
    domo_password=os.environ['DOJO_PASSWORD'],
    domo_username='jae@onyxreporting.com')

bs = DomoBootstrap(auth=full_auth)

feature_ls = await bs.get_features(return_raw=False)

pd.DataFrame(feature_ls[10:14])


,id,name,label,type,purchased,enabled
0,3440,admin-pages,,PREMIUM,True,True
1,3441,admin-slideshows,,PREMIUM,True,True
2,3442,admin-access-rights,,PREMIUM,True,True
3,3443,admin-tool-downloads,,PREMIUM,True,True
